# Create lighter dataset in json format for usage in viz

In [1]:
import pandas as pd
import numpy as np
import json
# imprt data from the large dataset
data_path = "data/aggregated_parlamentarier_nested.json"
df = pd.read_json(data_path)

In [35]:
data_branche = pd.DataFrame(columns=['branche', 'sub_branche', 'party', 'name', 'wirksamkeit', 'geld']) #define what we want as fields
for row in range(len(df)):
    for i in range(len(df.iloc[row]['interessenbindungen'])):
        if df.iloc[row]['interessenbindungen'][i]['organisation']!=None:
            for j in range(len(df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'])):
                # Get these fields
                branche = df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'][j]['branche']['anzeige_name_fr']
                sub_branche = df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'][j]['anzeige_name_fr']
                party = df.iloc[row]['partei_name']
                name = df.iloc[row]['name_fr']
                wirksamkeit = df.iloc[row]['interessenbindungen'][i]['wirksamkeit']
                wirksamkeit_nb = 0
                geld = 0
                match wirksamkeit: #transform wirksamkeit to int
                    case 'tief':
                        wirksamkeit_nb=3
                    case 'mittel':
                        wirksamkeit_nb=1
                    case 'hoch':
                        wirksamkeit_nb=2
                for x in range(len(df.iloc[row]['interessenbindungen'][i]['verguetungen'])): #sum money received
                    geld+=df.iloc[row]['interessenbindungen'][i]['verguetungen'][x]['verguetung']
                data_branche.loc[len(data_branche)]= [branche, sub_branche, party, name, wirksamkeit_nb, geld]

## Intermediate results

In [29]:
display(data_branche)

,branche,sub_branche,party,name,wirksamkeit,geld
0,Politique intérieure / Economie publique,Cantons/Régions,Schweizerische Volkspartei,Thomas Jan de Courten,3,0
1,Transport,Logistique/Transport,Schweizerische Volkspartei,Thomas Jan de Courten,1,5
2,Politique intérieure / Economie publique,Politique en général,Schweizerische Volkspartei,Thomas Jan de Courten,1,0
3,Économie,Fonds de pension et de retraite,Schweizerische Volkspartei,Thomas Jan de Courten,1,1
4,Économie,Biotechnologie,Schweizerische Volkspartei,Thomas Jan de Courten,3,5
...,...,...,...,...,...,...
7128,Sport,Sport populaire,Sozialdemokratische Partei,Andrea Zryd,3,-1
7129,Sport,Sport populaire,Sozialdemokratische Partei,Andrea Zryd,3,-1
7130,Sport,Sport d'élite,Sozialdemokratische Partei,Andrea Zryd,3,1
7131,Sécurité sociale,Enfants/Jeunesse,Sozialdemokratische Partei,Andrea Zryd,3,0


In [42]:
#sum the money and the efficency (wirksamkeit int) per parlementary members
data_branche2 = data_branche.groupby(by = ["branche", "sub_branche", "party", "name"])[['wirksamkeit', 'geld']].sum()  
data_branche = data_branche2.reset_index(level=['branche', 'sub_branche', 'party', 'name'])

In [48]:
display(data_branche) # Check if data is as expected

,branche,sub_branche,party,name,wirksamkeit,geld
0,Agriculture,Agriculture en général,Die Mitte,Andreas Meier,3,0
1,Agriculture,Agriculture en général,Die Mitte,Benjamin Roduit,3,0
2,Agriculture,Agriculture en général,Die Mitte,Brigitte Maria Häberli-Koller,3,0
3,Agriculture,Agriculture en général,Die Mitte,Charles Juillard,6,0
4,Agriculture,Agriculture en général,Die Mitte,Christian Lohr,6,0
...,...,...,...,...,...,...
4711,Énergie,Fournisseur d'énergie,Sozialdemokratische Partei,Flavia Wasserfallen,1,89400
4712,Énergie,Fournisseur d'énergie,Sozialdemokratische Partei,Nadine Désirée Masshardt,3,0
4713,Énergie,Fournisseur d'énergie,Sozialdemokratische Partei,Roger Nordmann,5,200000
4714,Énergie,Industrie du gaz et du pétrole,Freisinnig-Demokratische Partei,Martin Schmid,4,10


## Write as dict the data in the a particular form

In [49]:
list_branche = []
for branche in data_branche['branche'].unique():
    list_sb = []
    for sous_branche in data_branche[data_branche['branche'] == branche]['sub_branche'].unique():
        list_party = []
        for party in data_branche[(data_branche['branche'] == branche) & (data_branche['sub_branche'] == sous_branche)]['party'].unique():
            list_people = []
            for index, row in data_branche[(data_branche['branche'] == branche) & (data_branche['sub_branche'] == sous_branche) & (data_branche['party']==party)].iterrows():
                list_people.append({'name':row['name'], 'wirksamkeit':row['wirksamkeit'], 'party':row['party'], 'sub_branche':row['sub_branche'], 'branche':row['branche'], 'geld':row['geld']})
            list_party.append({'name':party, 'children':list_people})
        list_sb.append({'name':sous_branche, 'children':list_party})
    list_branche.append({'name':branche,'children':list_sb})
final_json = {'name': "lobby", 'children': list_branche}

In [50]:
# Write as json the dict
with open("data_final.json", "w") as final:
    json.dump(final_json, final)